# Vector Search vs Hybrid Search

In this cookbook, we'll explore the differences between pure vector search and hybrid search approaches that combine vector embeddings with metadata filtering. We'll see how structured metadata can dramatically improve search relevance and precision beyond what vector similarity alone can achieve.

When users search for products, documents, or other content, they often have specific attributes in mind. For example, a shopper might want "red dresses for summer occasions" or a researcher might need "papers on climate change published after 2020." Pure semantic search might miss these nuances, but metadata filtering allows you to combine the power of vector search with explicit attribute filtering.

Like always, we'll focus on data-driven approaches to measure and improve retrieval performance. 

## Requirements

Before starting, ensure you have the following packages installed:

In [ ]:
%pip install lancedb datasets openai tqdm pandas pyarrow tantivy pylance

## Setup

Start by setting up the environment and necessary libraries:

In [2]:
import getpass
import lancedb
import openai
from datasets import load_dataset

openai.api_key = getpass.getpass('Enter your OpenAI API key: ')
# Set up LanceDB
db = lancedb.connect('./lancedb_ecommerce_demo')

## The Dataset

In this cookbook, we'll work with a product catalog dataset containing fashion items with structured metadata. The dataset includes:

- **Basic product information**: titles, descriptions, brands, and prices
- **Categorization**: categories, subcategories, and product types
- **Attributes**: structured characteristics like sleeve length, neckline, and fit
- **Materials and patterns**: fabric types and design patterns

Here's what our taxonomy structure looks like:

```json
{
    "taxonomy_map": {
        "Women": {
            "Tops": {
                "product_type": ["T-Shirts", "Blouses", "Sweaters", "Cardigans", "Tank Tops", "Hoodies", "Sweatshirts"],
                "attributes": {
                    "Sleeve Length": ["Sleeveless", "Short Sleeve", "3/4 Sleeve", "Long Sleeve"],
                    "Neckline": ["Crew Neck", "V-Neck", "Turtleneck", "Scoop Neck", "Cowl Neck"],
                    "Fit": ["Regular", "Slim", "Oversized", "Cropped"]
                }
            },
            "Bottoms": {
                "product_type": ["Pants", "Jeans", "Shorts", "Skirts", "Leggings"],
                "attributes": {
                    // Additional attributes...
                }
            }
        }
    }
}
```

Having well-structured metadata enables more precise filtering and can significantly improve search relevance, especially for domain-specific applications where users have particular attributes in mind. This data might come from manual tagging by product managers or automated processes with LLMs. 

Let's first load the dataset from Huggingface: 

In [ ]:
from datasets import load_dataset
import pandas as pd

labelled_dataset = load_dataset("ivanleomk/labelled-ecommerce-taxonomy")["train"]
df = pd.DataFrame(labelled_dataset)
df["id"] = df["id"].astype(str)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/191 [00:00<?, ? examples/s]

## Generate Embeddings (OpenAI)

Now, let's create embeddings for our product descriptions. We'll use OpenAI's text-embedding-3-large model:

In [17]:
import numpy as np
from tqdm import tqdm

def batch_embed(texts, model="text-embedding-3-large"):
    batch_size = 100
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding..."):
        batch = texts[i:i+batch_size]
        response = openai.embeddings.create(model=model, input=batch)
        emb = [np.array(e.embedding, dtype='float32') for e in response.data]
        embeddings.extend(emb)
    return embeddings

df["embedding"] = batch_embed(df["description"].tolist())

Embedding...: 100%|██████████| 2/2 [00:07<00:00,  3.84s/it]


# Combine all text fields into a single searchable text field

We'll create a single text field that combines the product name, description, and category. This will allow us to perform a single search over all relevant text content:

In [33]:
df["searchable_text"] = df.apply(
    lambda row: " ".join([
        row["title"],
        row["description"],
        row["brand"],
        row["category"],
        row["subcategory"],
        row["product_type"],
        row["attributes"],
        row["material"],
        row["pattern"],
        row["occasions"],
    ]),
    axis=1
)
df["searchable_text"].head()

0    Lace Detail Sleeveless Top Elevate your casual...
1    Women's Classic Green T-Shirt Elevate your cas...
2    Striped Midi Skirt Add a touch of elegance to ...
3    Fila Women's Cropped Logo T-Shirt Elevate your...
4    Plaid Crop Top This chic plaid crop top featur...
Name: searchable_text, dtype: object

## Ingest data into LanceDB

We'll use LanceDB to store our product data and embeddings. LanceDB makes it easy to experiment, as it provides both vector and hybrid search capabilities within one single API.

In [37]:
import pyarrow as pa

table_schema = pa.schema(
    [
        pa.field("id", pa.string()),
        pa.field("description", pa.string()),
        pa.field("title", pa.string()),
        pa.field("brand", pa.string()),
        pa.field("category", pa.string()),
        pa.field("subcategory", pa.string()),
        pa.field("product_type", pa.string()),
        pa.field("attributes", pa.string()),
        pa.field("material", pa.string()),
        pa.field("pattern", pa.string()),
        pa.field("price", pa.float64()),
        pa.field("occasions", pa.string()),
        pa.field(
            "embedding", pa.list_(pa.float32(), 3072)
        ),  # size depends on your model!!
        pa.field("searchable_text", pa.string()),
    ]
)

# Drop unused columns
df_ = df.drop(columns=["image"])

# Create table + upload data
if "products" in db.table_names():
    tbl = db.open_table("products")
else:
    tbl = db.create_table("products", data=df_, schema=table_schema, mode="overwrite")

tbl.create_fts_index("searchable_text", replace=True)


## Generating Synthetic Data

When you don't have production data to start with, you can generate synthetic data to simulate a real-world scenario. We already have the 'output', which is the clothing item we just embedded. We now want to generate synthetic queries that would be relevant to the clothing item. 

In this case, we'll use GPT-4 to generate realistic user queries that would naturally lead to each product in our catalog. This gives us query-product pairs where we know the ground truth relevance.

In [19]:
import random
from openai import OpenAI
from tqdm import tqdm

# Initialize OpenAI client
client = OpenAI(api_key=openai.api_key)

# Define query types to generate variety
query_types = [
    "Basic search for specific item",
    "Search with price constraint",
    "Search for specific occasion",
    "Search with material preference",
    "Search with style/attribute preference"
]

def generate_synthetic_query(item):
    """Generate a realistic search query for a clothing item"""

    # Select a random query type
    query_type = random.choice(query_types)

    # Create prompt for the LLM
    prompt = f"""
    Generate a realistic search query that would lead someone to find this specific clothing item:

    Item Details:
    - Title: {item["title"]}
    - Description: {item["description"]}
    - Category: {item["category"]}
    - Subcategory: {item["subcategory"]}
    - Product Type: {item["product_type"]}
    - Price: ${item["price"]}
    - Material: {item["material"]}
    - Attributes: {item["attributes"]}
    - Occasions: {item["occasions"]}

    The query should be in a conversational tone, about 10-20 words, and focus on a {query_type.lower()}.
    Don't mention the exact product name, but include specific details that would make this item a perfect match.

    Example: For a $120 silk blouse with long sleeves, a query might be:
    "Looking for an elegant silk top with long sleeves for work, under $150"
    """

    # Generate query using OpenAI
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates realistic shopping queries."},
            {"role": "user", "content": prompt}
        ]
    )

    # Extract the generated query
    query = response.choices[0].message.content.strip().strip('"')

    return {"query": query, **item}

# Generate queries
synthetic_queries = []
for item in tqdm(labelled_dataset, desc="Generating queries"):
    query_data = generate_synthetic_query(item)
    synthetic_queries.append(query_data)

Generating queries: 100%|██████████| 191/191 [02:17<00:00,  1.39it/s]


Let's visualize what this looks like: 

In [20]:
from rich import print

print(synthetic_queries[0])

{
    'query': 'Searching for a sleeveless top with lace detailing at the neckline for casual outings and dinner 
dates.',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x13E0BB230>,
    'title': 'Lace Detail Sleeveless Top',
    'brand': 'H&M',
    'description': "Elevate your casual wardrobe with this elegant sleeveless top featuring intricate lace 
detailing at the neckline. Perfect for both day and night, it's crafted from a soft, breathable fabric for all-day 
comfort.",
    'category': 'Women',
    'subcategory': 'Tops',
    'product_type': 'Tank Tops',
    'attributes': '[{"name": "Sleeve Length", "value": "Sleeveless"}, {"name": "Neckline", "value": "Crew Neck"}]',
    'material': 'Cotton',
    'pattern': 'Solid',
    'id': 1,
    'price': 181.04,
    'occasions': '["Everyday Wear", "Casual Outings", "Smart Casual", "Dinner Dates", "Partywear"]'
}

## Hybrid Search on LanceDB

LanceDB makes it easy to combine vector search with full-text search in a single query. Let's see how this works with a practical example:

In [40]:
text_query = "dress for wedding guests"
vector_query = openai.embeddings.create(model="text-embedding-3-large", input=text_query).data[0].embedding

results = tbl.search(query_type="hybrid") \
    .text(text_query) \
    .vector(vector_query) \
    .limit(5) \
    .to_pandas()

results

,title,brand,description,category,subcategory,product_type,attributes,material,pattern,id,price,occasions,embedding,searchable_text,_relevance_score
0,Women's White Sleeveless Top with Skirt,H&M,This elegant white sleeveless top pairs perfec...,Women,Tops,Tank Tops,"[{""name"": ""Sleeve Length"", ""value"": ""Sleeveles...",Cotton,Solid,78,262.91,"[""Everyday Wear"", ""Casual Outings"", ""Semi-Form...","[-0.034495406, -0.029635048, -0.014213263, 0.0...",Women's White Sleeveless Top with Skirt This e...,0.032002
1,Lace Sleeve Top,Zara,This elegant top features delicate lace sleeve...,Women,Tops,Blouses,"[{""name"": ""Sleeve Length"", ""value"": ""Long Slee...",Polyester,Solid,154,101.15,"[""Everyday Wear"", ""Casual Outings"", ""Office We...","[-0.04646802, -0.0003945035, -0.022669518, 0.0...",Lace Sleeve Top This elegant top features deli...,0.032002
2,Women's Lace Blouse,Zara,Elevate your wardrobe with this elegant lace b...,Women,Tops,Blouses,"[{""name"": ""Sleeve Length"", ""value"": ""3/4 Sleev...",Polyester,Floral,14,88.59,"[""Casual Outings"", ""Formal Events"", ""Business ...","[-0.054958005, -0.000808206, -0.021859447, 0.0...",Women's Lace Blouse Elevate your wardrobe with...,0.016393
3,Women's Sleeveless Red Tank Dress,Diesel,This vibrant red tank dress features a sleevel...,Women,Dresses,Casual Dresses,"[{""name"": ""Sleeve Length"", ""value"": ""Sleeveles...",Cotton,Solid,134,281.24,"[""Everyday Wear"", ""Casual Outings"", ""Office We...","[-0.043403123, 0.004891483, -0.013913552, 0.03...",Women's Sleeveless Red Tank Dress This vibrant...,0.016393
4,Eyelet Button-Down Vest,Anthropologie,This charming eyelet vest is perfect for addin...,Women,Outerwear,Vests,"[{""name"": ""Length"", ""value"": ""Cropped""}]",Cotton,Floral,8,212.97,"[""Everyday Wear"", ""Casual Outings"", ""Semi-Form...","[-0.03877546, 0.021961322, -0.019559303, -0.00...",Eyelet Button-Down Vest This charming eyelet v...,0.015625


## Implementing Different Search Methods

To properly compare different search approaches, we'll implement three search functions:

In [72]:
import re

def sanitize_query(query):
    # Remove characters that break LanceDB FTS queries
    return re.sub(r"['\"\\]", "", query)

def search_semantic(tbl, query, embedding, k=5):
    return tbl.search(embedding).limit(k).to_pandas()["id"].tolist()

def search_lexical(tbl, query, k=5):
    # BM25 over description field
    return tbl.search(query=sanitize_query(query), query_type="fts").limit(k).to_pandas()["id"].tolist()

def search_hybrid(tbl, query, embedding, k=5):
    # Blends vector and BM25
    return tbl.search(query_type="hybrid").text(sanitize_query(query)).vector(embedding).limit(k).to_pandas()["id"].tolist()

These functions provide a clean interface for our three search methods:

Semantic search: Uses only vector embeddings to find similar products
Lexical search: Uses only BM25 text matching (similar to what traditional search engines use)
Hybrid search: Combines both approaches for potentially better results

Note that we sanitize the query text to remove characters that might break the full-text search functionality. This is an important preprocessing step when working with user-generated queries.

## Evaluation Metrics

To objectively compare our search methods, we'll use two standard information retrieval metrics:

1. **Recall**: The proportion of relevant items successfully retrieved
2. **Mean Reciprocal Rank (MRR)**: How high relevant items appear in our results

In [75]:
def recall(retrieved, expected):
    return float(len(set(retrieved).intersection(set(expected)))) / len(expected)

def mrr(retrieved, expected):
    # expected: list of relevant document ids (strings)
    for rank, doc_id in enumerate(retrieved, 1):
        if doc_id in expected:
            return 1.0 / rank
    return 0.0

def evaluate_search(tbl, queries, expected_ids, embeddings, k=5):
    metrics = dict(semantic=[], lexical=[], hybrid=[])
    for idx, query in enumerate(tqdm(queries, desc="Evaluating...")):
        eid = expected_ids[idx]
        emb = embeddings[idx]
        metrics["semantic"].append({
            "recall": recall(search_semantic(tbl, query, emb, k), eid),
            "mrr": mrr(search_semantic(tbl, query, emb, k), eid)
        })
        metrics["lexical"].append({
            "recall": recall(search_lexical(tbl, query, k), eid),
            "mrr": mrr(search_lexical(tbl, query, k), eid)
        })
        metrics["hybrid"].append({
            "recall": recall(search_hybrid(tbl, query, emb, k), eid),
            "mrr": mrr(search_hybrid(tbl, query, emb, k), eid)
        })
    return metrics

The evaluate_search function runs all three search methods on each query and calculates both metrics. This gives us a nice view of how each method performs across our test set.

# Prepare evaluation data

In [76]:
queries = [item["query"] for item in synthetic_queries]
expected_ids = [[str(item["id"])] for item in synthetic_queries]
query_embeddings = batch_embed(queries)  # for fair test, encode queries w/same embedding model

Embedding...: 100%|██████████| 2/2 [00:03<00:00,  1.85s/it]


## Run the Experiment

Now we can run the experiments. The code does the following:

1. Tests each search method with different numbers of results (k=3, 5, and 10)
2. Aggregates the metrics by calculating the mean recall and MRR for each method
3. Organizes the results in a DataFrame for easy comparison

In [78]:
k_values = [3, 5, 10]
results = []

for k in k_values:
    metrics = evaluate_search(tbl, queries, expected_ids, query_embeddings, k=k)
    import numpy as np
    def aggregate_metrics(metrics):
        return {m: {"recall": np.mean([x["recall"] for x in v]),
                    "mrr": np.mean([x["mrr"] for x in v])} for m, v in metrics.items()}
    summary = aggregate_metrics(metrics)
    for method, vals in summary.items():
        results.append({"k": k, "method": method, "recall": vals["recall"], "mrr": vals["mrr"]})

results_df = pd.DataFrame(results)
print(results_df)

Evaluating...: 100%|██████████| 191/191 [00:03<00:00, 51.79it/s]

    k    method    recall       mrr
0   3  semantic  0.905759  0.815881
1   3   lexical  0.937173  0.815009
2   3    hybrid  0.916230  0.848168
3   5  semantic  0.937173  0.822949
4   5   lexical  0.968586  0.822339
5   5    hybrid  0.947644  0.860297
6  10  semantic  0.973822  0.827988
7  10   lexical  0.984293  0.824447
8  10    hybrid  0.989529  0.868476


## Conclusion

Our evaluation demonstrates that hybrid search consistently outperforms both pure vector search and lexical search across all tested k values. Key findings:

- Hybrid search achieves the highest MRR scores, showing that combining semantic understanding with keyword matching places relevant results higher in the result list.
- Lexical search performs surprisingly well on recall, reminding us that traditional keyword approaches remain valuable for explicit queries.
- Vector search provides a solid baseline but benefits significantly from the precision that text matching adds.

As k increases, recall improves across all methods, but hybrid search maintains its advantage in ranking relevant items higher. These results highlight that the best search approach depends on your specific data and user query patterns. For product search where users combine concepts ("casual") with attributes ("red"), hybrid search offers clear advantages.

I hope this analysis helps you make informed decisions about the best approach for your own use case. Remember to:

1. Test multiple retrieval strategies on your specific data
2. Measure performance with appropriate metrics
3. Consider the trade-offs between implementation complexity and performance gains

In the next cookbook, we'll be looking at evaluating tool calling!